# URL域名提取和JSON整理工具

此notebook用於從WARC/JSONL數據中提取唯一域名並整理成JSON格式，方便後續分析和使用。

## 功能特點
- 從URL中提取完整域名
- 統計每個域名的出現頻次
- 生成結構化的JSON輸出
- 支持多種輸出格式

In [29]:
import json
import pandas as pd
from urllib.parse import urlparse
from collections import Counter, defaultdict
import os
from datetime import datetime
import re

print("📚 已導入所需庫")

📚 已導入所需庫


In [30]:
# 數據加載函數
def load_jsonl_data(file_path):
    """從JSONL文件加載數據"""
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                if line.strip():
                    try:
                        data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"第{line_num}行JSON解析錯誤: {e}")
        return data
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
        return []
    except Exception as e:
        print(f"讀取文件時出錯: {e}")
        return []

# 指定數據文件路径
data_files = [
    r"output_all\extracted_content_CC-MAIN-20240612140424-20240612170424-00000.warc.jsonl",
    r"fineweb-zhtw\data\output_high_quality\clean_traditional_chinese.jsonl"
]

print("🔍 開始加載數據文件...")

all_data = []
for file_path in data_files:
    if os.path.exists(file_path):
        print(f"加載文件: {file_path}")
        file_data = load_jsonl_data(file_path)
        print(f"  └─ 成功加載 {len(file_data)} 條記錄")
        all_data.extend(file_data)
    else:
        print(f"文件不存在: {file_path}")

print(f"\n✅ 總共加載了 {len(all_data)} 條記錄")

🔍 開始加載數據文件...
加載文件: output_all\extracted_content_CC-MAIN-20240612140424-20240612170424-00000.warc.jsonl
  └─ 成功加載 3184 條記錄
加載文件: fineweb-zhtw\data\output_high_quality\clean_traditional_chinese.jsonl
  └─ 成功加載 8 條記錄

✅ 總共加載了 3192 條記錄


In [31]:
# 域名提取和處理類
class DomainExtractor:
    """智能域名提取器"""
    
    def __init__(self):
        self.domain_stats = defaultdict(lambda: {
            'count': 0,
            'urls': [],
            'tld': '',
            'subdomain_count': 0,
            'first_seen': None,
            'last_seen': None
        })
    
    def extract_domain(self, url):
        """從URL提取域名"""
        try:
            parsed = urlparse(url)
            domain = parsed.netloc.lower()
            
            # 移除端口號
            if ':' in domain:
                domain = domain.split(':')[0]
            
            # 移除www前綴（可選）
            if domain.startswith('www.'):
                domain = domain[4:]
            
            return domain
        except Exception as e:
            print(f"URL解析錯誤 {url}: {e}")
            return None
    
    def extract_tld(self, domain):
        """提取頂級域名"""
        if not domain or '.' not in domain:
            return ''
        return domain.split('.')[-1]
    
    def count_subdomains(self, domain):
        """計算子域名數量"""
        if not domain:
            return 0
        return domain.count('.')
    
    def process_urls(self, data):
        """處理所有URL並提取域名信息"""
        print("🔄 開始處理URL並提取域名...")
        
        processed_count = 0
        for i, record in enumerate(data):
            if i % 1000 == 0:
                print(f"  處理進度: {i}/{len(data)} ({i/len(data)*100:.1f}%)")
            
            url = record.get('url', '')
            if not url:
                continue
            
            domain = self.extract_domain(url)
            if not domain:
                continue
            
            # 更新域名統計
            stats = self.domain_stats[domain]
            stats['count'] += 1
            stats['tld'] = self.extract_tld(domain)
            stats['subdomain_count'] = self.count_subdomains(domain)
            
            # 記錄URL示例（最多保存5個）
            if len(stats['urls']) < 5:
                stats['urls'].append(url)
            
            # 記錄時間戳（如果有的話）
            timestamp = record.get('timestamp') or record.get('date') or datetime.now().isoformat()
            if stats['first_seen'] is None:
                stats['first_seen'] = timestamp
            stats['last_seen'] = timestamp
            
            processed_count += 1
        
        print(f"✅ 處理完成！共處理 {processed_count} 個URL")
        print(f"📊 發現 {len(self.domain_stats)} 個唯一域名")
        
        return self.domain_stats

# 初始化域名提取器
extractor = DomainExtractor()

# 處理數據
if all_data:
    domain_data = extractor.process_urls(all_data)
else:
    print("⚠️ 沒有數據可處理")

🔄 開始處理URL並提取域名...
  處理進度: 0/3192 (0.0%)
  處理進度: 1000/3192 (31.3%)
  處理進度: 2000/3192 (62.7%)
  處理進度: 3000/3192 (94.0%)
✅ 處理完成！共處理 3192 個URL
📊 發現 2954 個唯一域名


In [32]:
# JSON輸出生成器
class JSONOutputGenerator:
    """生成多種格式的JSON輸出"""
    
    def __init__(self, domain_data):
        self.domain_data = domain_data
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    def generate_simple_list(self):
        """生成簡單的域名列表"""
        domains = list(self.domain_data.keys())
        domains.sort()
        return {
            "metadata": {
                "generated_at": datetime.now().isoformat(),
                "total_domains": len(domains),
                "format": "simple_list"
            },
            "domains": domains
        }
    
    def generate_detailed_stats(self):
        """生成詳細的域名統計信息"""
        detailed_data = {}
        
        for domain, stats in self.domain_data.items():
            detailed_data[domain] = {
                "count": stats['count'],
                "tld": stats['tld'],
                "subdomain_count": stats['subdomain_count'],
                "sample_urls": stats['urls'],
                "first_seen": stats['first_seen'],
                "last_seen": stats['last_seen']
            }
        
        return {
            "metadata": {
                "generated_at": datetime.now().isoformat(),
                "total_domains": len(detailed_data),
                "total_urls_processed": sum(stats['count'] for stats in self.domain_data.values()),
                "format": "detailed_stats"
            },
            "domains": detailed_data
        }
    
    def generate_frequency_ranked(self):
        """生成按頻率排序的域名列表"""
        sorted_domains = sorted(
            self.domain_data.items(), 
            key=lambda x: x[1]['count'], 
            reverse=True
        )
        
        ranked_list = []
        for rank, (domain, stats) in enumerate(sorted_domains, 1):
            ranked_list.append({
                "rank": rank,
                "domain": domain,
                "count": stats['count'],
                "percentage": round(stats['count'] / sum(s['count'] for s in self.domain_data.values()) * 100, 2),
                "tld": stats['tld']
            })
        
        return {
            "metadata": {
                "generated_at": datetime.now().isoformat(),
                "total_domains": len(ranked_list),
                "ranking_criteria": "url_frequency",
                "format": "frequency_ranked"
            },
            "domains": ranked_list
        }
    
    def generate_tld_grouped(self):
        """按頂級域名分組的域名列表"""
        tld_groups = defaultdict(list)
        
        for domain, stats in self.domain_data.items():
            tld = stats['tld'] or 'no_tld'
            tld_groups[tld].append({
                "domain": domain,
                "count": stats['count']
            })
        
        # 對每個TLD組內的域名按頻率排序
        for tld in tld_groups:
            tld_groups[tld].sort(key=lambda x: x['count'], reverse=True)
        
        return {
            "metadata": {
                "generated_at": datetime.now().isoformat(),
                "total_domains": len(self.domain_data),
                "total_tlds": len(tld_groups),
                "format": "tld_grouped"
            },
            "tld_groups": dict(tld_groups)
        }
    
    def save_all_formats(self, output_dir="domain_extracts"):
        """保存所有格式的JSON文件"""
        os.makedirs(output_dir, exist_ok=True)
        
        formats = {
            "simple_list": self.generate_simple_list(),
            "detailed_stats": self.generate_detailed_stats(),
            "frequency_ranked": self.generate_frequency_ranked(),
            "tld_grouped": self.generate_tld_grouped()
        }
        
        saved_files = []
        for format_name, data in formats.items():
            filename = f"{format_name}_{self.timestamp}.json"
            filepath = os.path.join(output_dir, filename)
            
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            
            saved_files.append(filepath)
            print(f"💾 已保存: {filepath}")
        
        return saved_files

# 生成JSON輸出
if 'domain_data' in locals() and domain_data:
    print("📄 開始生成JSON輸出...")
    
    generator = JSONOutputGenerator(domain_data)
    
    # 顯示統計摘要
    print(f"\n📊 域名統計摘要:")
    print(f"  唯一域名總數: {len(domain_data):,}")
    
    # 按頻率顯示前10個域名
    top_domains = sorted(domain_data.items(), key=lambda x: x[1]['count'], reverse=True)[:10]
    print(f"\n🏆 前10個最頻繁域名:")
    for i, (domain, stats) in enumerate(top_domains, 1):
        print(f"  {i:2d}. {domain}: {stats['count']} 次")
    
    # TLD統計
    tld_counter = Counter(stats['tld'] for stats in domain_data.values())
    print(f"\n🌐 頂級域名分布 (前5):")
    for tld, count in tld_counter.most_common(5):
        print(f"  .{tld}: {count} 個域名")
    
    # 保存所有格式
    print(f"\n💾 正在保存JSON文件...")
    saved_files = generator.save_all_formats()
    
    print(f"\n✅ 完成！共生成 {len(saved_files)} 個JSON文件")
    
else:
    print("⚠️ 沒有域名數據可生成JSON")

📄 開始生成JSON輸出...

📊 域名統計摘要:
  唯一域名總數: 2,954

🏆 前10個最頻繁域名:
   1. cujasweb.univ-paris1.fr: 5 次
   2. licitacoeseb.9rm.eb.mil.br: 4 次
   3. luna.library.cmu.edu: 4 次
   4. siganus.php.xdomain.jp: 4 次
   5. imagemagick.org: 4 次
   6. business.kanerepublican.com: 3 次
   7. collections.artmuseum.utoronto.ca: 3 次
   8. forum.artofwar.net.ru: 3 次
   9. ftp.cpan.org: 3 次
  10. ftp.mozilla.org: 3 次

🌐 頂級域名分布 (前5):
  .com: 1421 個域名
  .ru: 185 個域名
  .org: 138 個域名
  .net: 135 個域名
  .cn: 113 個域名

💾 正在保存JSON文件...
💾 已保存: domain_extracts\simple_list_20250717_162329.json
💾 已保存: domain_extracts\detailed_stats_20250717_162329.json
💾 已保存: domain_extracts\frequency_ranked_20250717_162329.json
💾 已保存: domain_extracts\tld_grouped_20250717_162329.json

✅ 完成！共生成 4 個JSON文件


In [33]:
# JSON數據預覽和驗證
def preview_json_formats():
    """預覽不同格式的JSON結構"""
    if 'generator' not in locals() or not domain_data:
        print("⚠️ 請先運行域名提取和JSON生成")
        return
    
    print("🔍 JSON格式預覽:")
    print("=" * 50)
    
    # 1. 簡單列表格式預覽
    simple_data = generator.generate_simple_list()
    print("📋 1. 簡單列表格式 (simple_list.json):")
    print(f"  └─ 包含 {simple_data['metadata']['total_domains']} 個域名")
    print("  └─ 結構預覽:")
    preview_simple = {
        "metadata": simple_data['metadata'],
        "domains": simple_data['domains'][:3] + ["..."] if len(simple_data['domains']) > 3 else simple_data['domains']
    }
    print(json.dumps(preview_simple, ensure_ascii=False, indent=4))
    
    print("\n" + "-" * 50)
    
    # 2. 詳細統計格式預覽
    detailed_data = generator.generate_detailed_stats()
    print("📊 2. 詳細統計格式 (detailed_stats.json):")
    print(f"  └─ 包含詳細統計信息")
    print("  └─ 結構預覽:")
    first_domain = list(detailed_data['domains'].keys())[0]
    preview_detailed = {
        "metadata": detailed_data['metadata'],
        "domains": {
            first_domain: detailed_data['domains'][first_domain],
            "...": "更多域名數據"
        }
    }
    print(json.dumps(preview_detailed, ensure_ascii=False, indent=4))
    
    print("\n" + "-" * 50)
    
    # 3. 頻率排序格式預覽
    ranked_data = generator.generate_frequency_ranked()
    print("🏆 3. 頻率排序格式 (frequency_ranked.json):")
    print(f"  └─ 按URL出現頻率排序")
    print("  └─ 結構預覽:")
    preview_ranked = {
        "metadata": ranked_data['metadata'],
        "domains": ranked_data['domains'][:3] + [{"...": "更多排序數據"}] if len(ranked_data['domains']) > 3 else ranked_data['domains']
    }
    print(json.dumps(preview_ranked, ensure_ascii=False, indent=4))
    
    print("\n" + "-" * 50)
    
    # 4. TLD分組格式預覽
    tld_data = generator.generate_tld_grouped()
    print("🌐 4. TLD分組格式 (tld_grouped.json):")
    print(f"  └─ 按頂級域名分組")
    print("  └─ 結構預覽:")
    first_tld = list(tld_data['tld_groups'].keys())[0]
    preview_tld = {
        "metadata": tld_data['metadata'],
        "tld_groups": {
            first_tld: tld_data['tld_groups'][first_tld][:2] + [{"...": "更多域名"}] if len(tld_data['tld_groups'][first_tld]) > 2 else tld_data['tld_groups'][first_tld],
            "...": "更多TLD分組"
        }
    }
    print(json.dumps(preview_tld, ensure_ascii=False, indent=4))

# 運行預覽
if 'domain_data' in locals() and domain_data:
    preview_json_formats()
    
    print(f"\n📁 生成的JSON文件用途說明:")
    print("=" * 50)
    print("1️⃣  simple_list.json:")
    print("   └─ 純域名列表，適合快速查看和導入其他工具")
    print("2️⃣  detailed_stats.json:")
    print("   └─ 完整統計信息，包含URL示例、出現次數等")
    print("3️⃣  frequency_ranked.json:")
    print("   └─ 按熱門程度排序，方便識別主要域名來源")
    print("4️⃣  tld_grouped.json:")
    print("   └─ 按網域類型分組，便於分析域名分布特徵")
    
    print(f"\n💡 使用建議:")
    print("  • 用於白名單/黑名單管理")
    print("  • 域名信譽分析") 
    print("  • 數據來源追踪")
    print("  • 版權風險評估")
    print("  • 網站分類和過濾")

else:
    print("⚠️ 請先運行前面的數據加載和處理步驟")

⚠️ 請先運行域名提取和JSON生成

📁 生成的JSON文件用途說明:
1️⃣  simple_list.json:
   └─ 純域名列表，適合快速查看和導入其他工具
2️⃣  detailed_stats.json:
   └─ 完整統計信息，包含URL示例、出現次數等
3️⃣  frequency_ranked.json:
   └─ 按熱門程度排序，方便識別主要域名來源
4️⃣  tld_grouped.json:
   └─ 按網域類型分組，便於分析域名分布特徵

💡 使用建議:
  • 用於白名單/黑名單管理
  • 域名信譽分析
  • 數據來源追踪
  • 版權風險評估
  • 網站分類和過濾
